In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.chdir(r"D:\PythonExercise")

In [2]:
df = pd.read_excel(r"Online Retail.xlsx")
df.head()
df1 = df

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [4]:
df1['Country'].nunique()

38

In [5]:
df1['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [6]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID',ascending=False)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [7]:
a=customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index()

In [8]:
b=customer_country.groupby(['Country'])['CustomerID'].aggregate('count')

In [9]:
b.sort_values(ascending=False)

Country
United Kingdom          3950
Germany                   95
France                    87
Spain                     31
Belgium                   25
Switzerland               21
Portugal                  19
Italy                     15
Finland                   12
Austria                   11
Norway                    10
Channel Islands            9
Netherlands                9
Denmark                    9
Australia                  9
Cyprus                     8
Sweden                     8
Japan                      8
Poland                     6
Canada                     4
Greece                     4
Unspecified                4
USA                        4
Israel                     4
EIRE                       3
Malta                      2
Bahrain                    2
United Arab Emirates       2
Iceland                    1
Lebanon                    1
Lithuania                  1
European Community         1
Czech Republic             1
RSA                        1
Saudi 

In [11]:
df1=df1.loc[df1['Country']=='United Kingdom']

In [12]:
df1.isnull().sum(axis=0)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [13]:
df1=df1[pd.notnull(df1['CustomerID'])]

In [14]:
df1.isnull().sum(axis=0)

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [15]:
df1['Quantity'].min()

-80995

In [16]:
df1=df1[df1['Quantity']>0]
df1.shape

(354345, 8)

In [17]:
#check unique value for each column
def unique_counts(df):
    for i in df1.columns:
        count=df1[i].nunique()
        print(i,": ",count)
unique_counts(df1)
      

InvoiceNo :  16649
StockCode :  3645
Description :  3844
Quantity :  294
InvoiceDate :  15615
UnitPrice :  403
CustomerID :  3921
Country :  1


In [18]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']

In [19]:
import datetime as dt
NOW=dt.datetime(2011,12,10)
df1['InvoiceDate']=pd.to_datetime(df1['InvoiceDate'])

In [20]:
#RFM
rfmTable=df1.groupby('CustomerID').agg({'InvoiceDate':lambda x: (NOW-x.max()).days,'InvoiceNo': lambda x: len(x),'TotalPrice':lambda x:x.sum()})

In [22]:
rfmTable['InvoiceDate']=rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate':'recency',
                         'InvoiceNo':'frequency',
                         'TotalPrice':'monetary_value'},inplace=True)

In [23]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34


In [24]:
quantiles=rfmTable.quantile(q=[0.25,0.5,0.75])


In [27]:
quantiles=quantiles.to_dict()

In [28]:
quantiles

{'recency': {0.25: 17.0, 0.5: 50.0, 0.75: 142.0},
 'frequency': {0.25: 17.0, 0.5: 41.0, 0.75: 99.0},
 'monetary_value': {0.25: 300.03999999999996,
  0.5: 651.8199999999999,
  0.75: 1575.89}}

In [57]:
quantiles['recency'][0.25]

17.0

In [29]:
segmentrfm=rfmTable

In [31]:
def RScore(x,p,d):
    if x<=d[p][0.25]:
        return 1
    elif x<=d[p][0.5]:
        return 2
    elif x<=d[p][0.75]:
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [53]:
segmentrfm['recency'].apply(RScore)

TypeError: RScore() missing 2 required positional arguments: 'p' and 'd'

In [32]:
segmentrfm['r_quartile']=segmentrfm['recency'].apply(RScore,args=('recency',quantiles,))

In [37]:
segmentrfm['f_quartile']=segmentrfm['frequency'].apply(FMScore,args=('frequency',quantiles,))

In [41]:
segmentrfm['m_quartile'] = segmentrfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))

In [42]:
segmentrfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346.0,325,1,77183.60,4,4,1
12747.0,2,103,4196.01,1,1,1
12748.0,0,4596,33719.73,1,1,1
12749.0,3,199,4090.88,1,1,1
12820.0,3,59,942.34,1,2,2


In [45]:
segmentrfm['RFMScore']=segmentrfm['r_quartile'].map(str)+segmentrfm['f_quartile'].map(str)+segmentrfm['m_quartile'].map(str)

In [46]:
segmentrfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,4,4,1,441
12747.0,2,103,4196.01,1,1,1,111
12748.0,0,4596,33719.73,1,1,1,111
12749.0,3,199,4090.88,1,1,1,111
12820.0,3,59,942.34,1,2,2,122


In [47]:
segmentrfm[segmentrfm['RFMScore']=='111'].sort_values('monetary_value',ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111
15769.0,7,130,56252.72,1,1,1,111
